In [1]:
import sys
import os
import csv

In [2]:
# Armar diccionarios para guardar info
packs_sent = {}; packs_rcv = {}
delays = {}; delays_stats = {}
duty_cycle_lines = {};
duty_cycle = {};

In [3]:
# Contar cantidad de paquetes enviados y recibidos
def count_packs_sent_rcv(line):
    if "Sent network uptime" in line:
        id = line.split(":")[1]
        packs_sent[id] += 1
    if "Received from" in line:
        id = str(int(line.split(":0:0:")[1][0], 16)) # Extraer id y pasar a decimal
        packs_rcv[id] += 1

In [4]:
# Calcular estadística de los delays
def calculate_delays_stats(line):
    if "Received from" in line:
        id = str(int(line.split(":0:0:")[1][0], 16)) # Extraer id y pasar a decimal
        delay = int(line.split("latency ")[1][0])
        # Actualizar delay mínimo
        if delay < delays[id]['min']:
            delays[id]['min'] = delay
        # Actualizar delay máximo
        if delay > delays[id]['max']:
            delays[id]['max'] = delay
        # Actualizar delay promedio
        if delays[id]['avg'] == -1:
            delays[id]['avg'] = delay
        else:
            delays[id]['avg'] = (delays[id]['avg'] + delay) /2

In [5]:
# Redondear valores promedio de los delays
def round_delay_avg(delays):
    for id, stats in delays.items():
        stats['avg'] = round(stats['avg'], 3)
        delays[id] = stats

In [6]:
# Cargar el diccionario auxiliar duty_cycle_lines con la última línea de consumo de cada nodo
def load_duty_cycle_lines(line):
    if 'CPU' in line:
        id = line.split(":")[1] 
        duty_cycle_lines[id]['line_1'] = line
    if 'Radio LISTEN' in line:
        id = line.split(":")[1]
        duty_cycle_lines[id]['line_2'] = line

In [7]:
# Calcular el consumo de los nodos
def calculate_duty_cycle():
    for id, lines in duty_cycle_lines.items(): # id : {'line_1' : "LINEA 1..." , 'line_2' : "LINEA 2..."}
        for nro, line in lines.items(): # line_i : "LINEA i..."
            # Obtener claves según línea a procesar
            keys = list()
            if nro == 'line_1':
                keys = ['CPU', 'LPM', 'DEEP LPM']
            else: 
                keys = ['Radio LISTEN', 'TRANSMIT', 'OFF']
            # Obtener valores numéricos
            numbers = [int(s) for s in line.split() if s.isdigit()]
            # Completar diccionario de consumos
            for i in range(len(keys)):
                duty_cycle[id][keys[i]] = numbers[i]

In [8]:
# Parsear archivo de log
def process_log(file_path, motes_q, topo):
    # SE ASUME UN SOLO EMISOR (MOTE 1) EN TODAS LAS TOPOLOGÍAS
    motes_extra = 2 if topo == "E" else 1 # Se agrega nodo adicional por nodo aislado si es estrella
        
    # El consumo también se mide para el nodo 1
    duty_cycle_lines["1"] = {'line_1' : "", 'line_2': ""}
    duty_cycle["1"] = {'CPU' : 0, 'LPM' : 0, 'DEEP LPM' : 0, 'Radio LISTEN' : 0, 'TRANSMIT' : 0, 'OFF' : 0}
    
    for id in range(2, motes_q + motes_extra): 
        
        packs_sent[str(id)] = 0
        packs_rcv[str(id)] = 0
        delays[str(id)] = {'min' : sys.maxsize, 'max' : -1, 'avg' : -1}
        duty_cycle_lines[str(id)] = {'line_1' : "", 'line_2': ""}
        duty_cycle[str(id)] = {'CPU' : 0, 'LPM' : 0, 'DEEP LPM' : 0, 'Radio LISTEN' : 0, 'TRANSMIT' : 0, 'OFF' : 0}
    
    with open(file_path) as fp:
        line = fp.readline()
        while line:
            # Cantidad paqs enviados y recibidos
            count_packs_sent_rcv(line)
            # Estadísticas de los delays
            calculate_delays_stats(line)
            # Cargar diccionario para calcular consumo
            load_duty_cycle_lines(line)
            line = fp.readline()
        # Redondear delays promedio
        round_delay_avg(delays)
        # Calcular consumo
        calculate_duty_cycle()

In [9]:
def get_dir_paths(path):
    dir_paths = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for directory in d:
            dir_paths.append(os.path.join(r, directory))

    return dir_paths

In [10]:
def get_files_paths(path):
    files_paths = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            files_paths.append(os.path.join(r, file))

    return files_paths

In [11]:
### USO MÚLTIPLES ARCHIVOS
path = 'logs\\log_motes_minimal\\'
dir_paths = get_dir_paths(path)

motes_q_ant = -1

# Recorrer todas las carpetas
for dp in dir_paths:
    files_paths = get_files_paths(dp)
    # Recorrer todos los archivos para esa carpeta
    for fp in files_paths:
        print(fp)
        
        # E5_slot7_int3_testlog2020-02-22 23_24_27.404165
        motes_q = int(fp.split("\\")[3].split("_")[0][1:])  # Cantidad de nodos
        topo = fp.split("\\")[3].split("_")[0][0]           # Topología
        
        if motes_q != motes_q_ant:
            # Armar diccionarios para guardar info
            packs_sent = {}; packs_rcv = {}
            delays = {}; delays_stats = {}
            duty_cycle_lines = {};
            duty_cycle = {};
        
        # Procesar log
        process_log(fp, motes_q, topo)
        
        # Armar columnas para archivo CSV
        columns = ["pkt_sent", "pkt_rcv", "delays", "duty_cycle"]
        final_columns = list()
        ids = list(duty_cycle.keys())
        # Agregar ids a columnas
        for c in columns:
            for i in ids:
                if int(i) > 1 or c == "duty_cycle": # No crear columnas para nodo 1, salvo para consumo
                    # Para columna "delays", agregar además "avg", "max", "min"
                    if c == "delays":
                        final_columns.append(c + '_' + i + '_' + "min")
                        final_columns.append(c + '_' + i + '_' + "max")
                        final_columns.append(c + '_' + i + '_' + "avg")
                    elif c == "duty_cycle":
                        dcs = ['CPU', 'LPM', 'DEEP LPM', 'Radio LISTEN', 'TRANSMIT', 'OFF']
                        for dc in dcs:
                            final_columns.append(c + '_' + i + '_' + dc)
                    else:
                        final_columns.append(c + '_' + i)
            
        # Agregar columnas con info de la simulación
        # E5_slot7_int3_testlog2020-02-22 23_24_27.404165
        simulation_info = ["slotframe", "intensidad", "timestamp"]
        final_columns = simulation_info + final_columns        
        
        # Escribir resultados en archivo
        out_path = "results_" + topo + "_" + str(motes_q) + ".csv"
        # Crear o agregar en archivo de salida
        if os.path.exists(out_path):
            append_write = 'a' # agregar si el archivo existe
        else:
            append_write = 'w' # crear un nuevo archivo si no existe
        
        with open(out_path, append_write, newline='') as file:
            writer = csv.writer(file)
            if append_write == 'w': # Si se acaba de crear el archivo
                writer.writerow(final_columns) # Escribir encabezado
            data = list()
            # logs\log_motes\TSCH_E10\E5_slot7_int3_testlog2020-02-22 23_24_27.404165
            # Cargar info de la simulación
            data.append(fp.split("\\")[3].split("slot")[1].split("_int")[0])        # slotframe
            data.append(fp.split("\\")[3].split("int")[1].split("_testlog")[0])     # intensidad
            data.append(fp.split("\\")[3].split("testlog")[1])                      # timestamp
            # Cargar info de paquetes enviados
            for id, ps in packs_sent.items():
                # data.append(id + "_" + str(ps))                                 # paquetes enviados
                data.append(str(ps))                                              # paquetes enviados
            # Cargar info de paquetes recibidos
            for id, pr in packs_rcv.items():
                # data.append(id + "_" + str(pr))                                 # paquetes recibidos
                data.append(str(pr))                                              # paquetes recibidos
            # Cargar info de delays
            for id, stats in delays.items():
                for k, s in stats.items():
                    # data.append(id + "_" + k + "_" + str(s))                    # estadística de delays
                    data.append(str(s))                                           # estadística de delays
            # Cargar info de consumo
            for id, dcs in duty_cycle.items():
                for k, dc in dcs.items():
                    # data.append(id + "_" + k + "_" + str(dc))                   # consumo
                    data.append(str(dc))                                          # consumo
            # Escribir línea con datos
            writer.writerow(data)

logs\log_motes_minimal\estrella\E10_slot17_int19_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot17_int3_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot17_int8_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot27_int19_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot27_int3_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot27_int8_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot7_int19_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot7_int3_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E10_slot7_int8_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E5_slot17_int19_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E5_slot17_int3_testlog2020-03-17 22_50_22.958319
logs\log_motes_minimal\estrella\E5_slot17_int8_testlog2020-03-17 22_50_22.958319
logs\log_motes_min

logs\log_motes_minimal\lineal\L9_slot7_int8_testlog2020-03-17 22_50_22.958319
